<a href="https://colab.research.google.com/github/ankitbhutani1090/value_of_data/blob/main/notebooks/SMM5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import copy
import matplotlib.pyplot as plt
import os
import math
import seaborn as sns
from scipy.special import erfinv  # for lognormal quantile grid
import statsmodels.api as sm


project_dir = '/content/drive/MyDrive/Data_GDP_OV'

## Functions

### Diagnostics Functions

In [3]:
from dataclasses import dataclass

@dataclass
class DiagLite:
    # --- VFI (solve) ---
    flow_floor_hits: int = 0          # # times A - γ·ω (or (1-γ·ω)A) was floored at PROD_FLOOR
    flow_total_vfi: int = 0           # total # flow checks (≈ Nw per VFI iteration per a-node)
    den_floor_hits_vfi: int = 0       # # times omega denominator floored at DEN_FLOOR
    den_total_vfi: int = 0            # total # denominator evaluations in VFI (≈ Nw*Nk per iter per a-node)
    k_at_min_states: int = 0          # # states where policy picked k_min
    k_at_max_states: int = 0          # # states where policy picked k_max
    states_count: int = 0             # total # states considered for policy edges (≈ Nw per VFI iteration per a-node)
    nans_vfi: int = 0                 # # NaN/Inf incidents in VFI

    # --- Simulation ---
    real_floor_hits: int = 0          # # times A - γ·ψ² (or (1-γ·ψ²)A) was floored at PROD_FLOOR
    real_total_sim: int = 0           # total firm-years
    den_floor_hits_sim: int = 0       # # times omega denominator floored at DEN_FLOOR
    den_total_sim: int = 0            # total # denom evaluations in SIM (== firm-years)
    omega_at_min_sim: int = 0         # # times ω' hit lower grid bound
    omega_at_max_sim: int = 0         # # times ω' hit upper grid bound
    omega_updates_sim: int = 0        # total # ω updates (== firm-years)
    nans_sim: int = 0                 # # NaN/Inf incidents in SIM

    def report(self, label=""):
        msgs = []

        # VFI percentages
        if self.flow_total_vfi > 0 and self.flow_floor_hits > 0:
            pct = 100.0 * self.flow_floor_hits / self.flow_total_vfi
            msgs.append(f"[{label}] flow productivity floored: {pct:.2f}% "
                        f"({self.flow_floor_hits}/{self.flow_total_vfi})")
        if self.den_total_vfi > 0 and self.den_floor_hits_vfi > 0:
            pct = 100.0 * self.den_floor_hits_vfi / self.den_total_vfi
            msgs.append(f"[{label}] omega-denominator floored (VFI): {pct:.2f}% "
                        f"({self.den_floor_hits_vfi}/{self.den_total_vfi})")
        if self.states_count > 0:
            pct_min = 100.0 * self.k_at_min_states / self.states_count
            pct_max = 100.0 * self.k_at_max_states / self.states_count
            msgs.append(f"[{label}] policy grid hits: k_min={pct_min:.2f}%, "
                        f"k_max={pct_max:.2f}% (of {self.states_count} states)")
        if self.nans_vfi:
            msgs.append(f"[{label}] NaNs/Infs in VFI: {self.nans_vfi}")

        # SIM percentages
        if self.real_total_sim > 0 and self.real_floor_hits > 0:
            pct = 100.0 * self.real_floor_hits / self.real_total_sim
            msgs.append(f"[{label}] realized productivity floored: {pct:.2f}% "
                        f"({self.real_floor_hits}/{self.real_total_sim})")
        if self.den_total_sim > 0 and self.den_floor_hits_sim > 0:
            pct = 100.0 * self.den_floor_hits_sim / self.den_total_sim
            msgs.append(f"[{label}] omega-denominator floored (SIM): {pct:.2f}% "
                        f"({self.den_floor_hits_sim}/{self.den_total_sim})")
        if self.omega_updates_sim > 0 and (self.omega_at_min_sim or self.omega_at_max_sim):
            pct_min = 100.0 * self.omega_at_min_sim / self.omega_updates_sim
            pct_max = 100.0 * self.omega_at_max_sim / self.omega_updates_sim
            msgs.append(f"[{label}] omega bound hits (SIM): min={pct_min:.2f}%, "
                        f"max={pct_max:.2f}% (of {self.omega_updates_sim} updates)")
        if self.nans_sim:
            msgs.append(f"[{label}] NaNs/Infs in SIM: {self.nans_sim}")

        if msgs:
            print("\n".join(msgs))

### Other utilities

In [4]:
# ==========================
# Utilities
# ==========================
def lognormal_grid(mu_lnA, sd_lnA, N):
    """
    Quantile grid for A from LogNormal(mu, sd) using inverse-CDF of Normal.
    """
    qs = np.linspace(0.01, 0.99, N)
    lnA = mu_lnA + sd_lnA * np.sqrt(2.0) * erfinv(2 * qs - 1)
    return np.exp(lnA)

def softfloor(x, floor, k=50.0):
    """
    Smooth approximation of max(x, floor) using numerically stable softplus.
    """
    z = k * (x - floor)
    softplus = np.empty_like(z)

    pos = z > 0
    # For z > 0: softplus(z) = z + log1p(exp(-z)) (exp(-z) is tiny, safe)
    softplus[pos] = z[pos] + np.log1p(np.exp(-z[pos]))
    # For z <= 0: softplus(z) = log1p(exp(z)) (exp(z) <= 1, safe)
    softplus[~pos] = np.log1p(np.exp(z[~pos]))

    return floor + softplus / k

### VFI and Simulation functions

In [5]:
def productivity_term(a, varTerm, gamma, mode, w):
    """
    mode: additive or multiplicative
    Uses soft flooring to avoid non-differentiable kinks.
    """
    if mode == "additive":
        raw = a - gamma * varTerm - w
    elif mode == "multiplicative":
        raw = a * (1.0 - gamma * varTerm) - w
    else:
        raise ValueError("mode must be additive or multiplicative")

    # soft floor instead of hard max
    prod = softfloor(raw, PROD_FLOOR, k=10.0)
    return prod

def omega_next_from_rev(Rev, phi, tau, chi, Rev_M):
    """
    Implements: omega' = 1 / max(phi + tau + chi * log(max(Rev/Rev_M, eps)), DEN_FLOOR).
    Returns (omega_next, den_raw, den_floored)
    """
    ratio = np.maximum(Rev / Rev_M, RATIO_EPS)
    den   = phi + tau + chi * np.log(ratio)
    den_f = np.maximum(den, DEN_FLOOR)
    return 1.0 / den_f, den, den_f


# ==========================
# Grid builder (robust-ish)
# ==========================
def make_grids(mu_lnA, sd_lnA,
               N_a=9, N_w=60, N_k=80,
               alpha=1/3, r=0.04,
               phi=142.86, tau=312.04, chi=3.0, Rev_M=692.357,
               gamma=2.0, prod_mode="additive", w = 0.01,
               k_min=1e-6, logRev_cap=11, a_ref_q=0.99,
               omega_min = 1e-7, omega_max = 0.2):
    """
    Builds (a_grid, omega_grid, k_grid).
      - a_grid: lognormal quantiles
      - k_grid: geometric [k_min, k_max] based on myopic at worst-info omega ≈ 1/phi
      - omega_grid: geometric between model-based bounds implied by log link
    """
    a_grid = lognormal_grid(mu_lnA, sd_lnA, N_a)
    A_ref = np.quantile(a_grid, a_ref_q)

    # k_max from myopic formula at worst-info omega ≈ 1/phi, using selected productivity spec
    omega_worst = 1.0 / 2
    coeff_flow_worst = productivity_term(A_ref, omega_worst, gamma, prod_mode, w)
    base = (alpha / r) * coeff_flow_worst
    k_ref = base ** (1.0 / (1.0 - alpha))

    # conservative cap via data-driven Rev cap
    Rev_cap = float(np.exp(logRev_cap))
    # Use A_ref for rough cap: Rev ≈ A_ref * k^alpha <= Rev_cap
    k_cap = (Rev_cap / max(A_ref, 1e-12)) ** (1.0 / alpha)
    #print(f"kcap = {k_cap}, multiplier = {k_ref_multiplier * k_ref}")
    #k_max = float(min(k_ref_multiplier * k_ref, k_cap))
    k_max = k_cap
    print(f"k_max = {k_max}")
    k_grid = np.geomspace(k_min, k_max, N_k)  # dense near zero

    # omega bounds via log link
    # Best-info reference (use Rev from flow coeff at k_ref)
    Rev_ref = min(Rev_cap, coeff_flow_worst * (k_ref ** alpha))
    #omega_min, den_ref, den_ref_f = omega_next_from_rev(Rev_ref, phi, tau, chi, Rev_M)

    #omega_min = 0.0001 # Override omega_min

    # Worst-info reference (small revenue fraction)
    ratio_worst = 0.05
    Rev_worst   = ratio_worst * Rev_M
    #omega_max, den_worst, den_worst_f = omega_next_from_rev(Rev_worst, phi, tau, chi, Rev_M)

    #omega_max = 0.05 # Override omega_max

    omega_grid = np.geomspace(omega_min, omega_max, N_w)

    print(f"[Grids] A_ref(q={a_ref_q:.2f})={A_ref:.3g} | k_max={k_max:.3g}")
    print(f"[Grids] Rev_ref={Rev_ref:.3g} | omega_min={omega_min:.3e}")
    print(f"[Grids] worst Rev={Rev_worst:.3g} | omega_max={omega_max:.3e}")

    return a_grid, omega_grid, k_grid


# ==========================
# VFI (per a-node), vectorized
# ==========================
def solve_vfi_for_a(a, omega_grid, k_grid, alpha, r, phi, tau, chi, Rev_M,
                    gamma, prod_mode, w, diag: DiagLite):
    """
    Solves V(omega; a) and policy k*(omega; a) on grids via value function iteration.
    Transition uses expected-flow productivity (omega) for Rev in the mapping.
    """
    Nw, Nk = len(omega_grid), len(k_grid)
    V = np.zeros(Nw)
    policy_idx = np.zeros(Nw, dtype=int)

    for it in range(VFI_MAXIT):
        # Flow payoff for all states and k
        coeff_flow = productivity_term(a, omega_grid[:, None], gamma, prod_mode, w)  # (Nw,1)
        flow = coeff_flow * (k_grid[None, :] ** alpha) - r * k_grid[None, :]

        # Count how often flow coeff was floored (diagnostic prints only if >0 overall)
        if prod_mode == "additive":
            raw_flow = a - gamma * omega_grid[:, None] - w
        else:
            raw_flow = (1.0 - gamma * omega_grid[:, None]) * a - w
        diag.flow_floor_hits += int(np.count_nonzero(raw_flow < PROD_FLOOR))
        diag.flow_total_vfi  += raw_flow.size
        diag.states_count += Nw

        # Omega' from Rev using flow coeff (deterministic transition for VFI)
        Rev_flow = coeff_flow * (k_grid[None, :] ** alpha)  # (Nw,Nk)
        ratio = np.maximum(Rev_flow / Rev_M, RATIO_EPS)
        den_raw = phi + tau + chi * np.log(ratio)           # (Nw,Nk)
        den_f = np.maximum(den_raw, DEN_FLOOR)
        diag.den_floor_hits_vfi += int(np.count_nonzero(den_raw < DEN_FLOOR - 1e-12))
        diag.den_total_vfi      += den_raw.size
        omega_next = 1.0 / den_f

        # Interpolate V at omega_next (linear)
        idx = np.clip(np.searchsorted(omega_grid, omega_next, side='left') - 1, 0, Nw - 2)
        w_hi = omega_grid[idx + 1]
        w_lo = omega_grid[idx]
        weight = np.divide(omega_next - w_lo, w_hi - w_lo, out=np.zeros_like(omega_next), where=(w_hi > w_lo))
        V_next = (1.0 - weight) * V[idx] + weight * V[idx + 1]

        RHS = flow + BETA * V_next
        new_idx = np.argmax(RHS, axis=1)
        V_new = RHS[np.arange(Nw), new_idx]

        # policy boundary hits
        diag.k_at_min_states += int(np.count_nonzero(new_idx == 0))
        diag.k_at_max_states += int(np.count_nonzero(new_idx == Nk - 1))

        # NaN/Inf guard
        if not np.isfinite(V_new).all():
            diag.nans_vfi += 1
            V_new = np.where(np.isfinite(V_new), V_new, V)

        diff = np.max(np.abs(V_new - V))
        V = V_new
        policy_idx = new_idx

        if diff < VFI_TOL:
            break

    return V, policy_idx


def solve_policy(a_grid, omega_grid, k_grid, alpha, r, phi, tau, chi, Rev_M, gamma, prod_mode, w):
    diag = DiagLite()
    policies = []
    values = []
    for a in a_grid:
        V_a, pol_idx_a = solve_vfi_for_a(
            a, omega_grid, k_grid, alpha, r, phi, tau, chi, Rev_M,
            gamma, prod_mode, w, diag
        )
        policies.append(pol_idx_a)
        values.append(V_a)
    policies = np.stack(policies, axis=0)  # (Na, Nw)
    values = np.stack(values, axis=0)      # (Na, Nw)

    # Only print if something actually hit
    diag.report(label="VFI")
    return values, policies, diag


# ==========================
# Simulation -> DataFrame
# ==========================
def simulate_panel(N_firms, T, a_grid, omega_grid, k_grid, policies,
                   mu_lnA, sd_lnA, alpha, phi, tau, chi, Rev_M,
                   gamma, prod_mode, w, seed=2025):
    rng = np.random.default_rng(seed)
    # draw permanent A_i ~ LogNormal
    lnA = rng.normal(mu_lnA, sd_lnA, size=N_firms)
    A = np.exp(lnA)

    # init states (start from median omega)
    omega0 = float(np.median(omega_grid))
    omega = np.full(N_firms, omega0)

    # nearest-neighbor indexers
    def idx_a(a_i):
        return int(np.clip(np.searchsorted(a_grid, a_i, side='left'), 1, len(a_grid)-1) - 1)
    def idx_w(w_i):
        return int(np.clip(np.searchsorted(omega_grid, w_i, side='left'), 1, len(omega_grid)-1) - 1)

    rows = []
    diag = DiagLite()

    for t in range(T):
        for i in range(N_firms):
            a_i = A[i]
            iw  = idx_w(omega[i])
            ia  = idx_a(a_i)
            k   = k_grid[policies[ia, iw]]

            # realized productivity with psi^2
            z   = rng.standard_normal()
            psi2 = omega[i] * (z ** 2)
            coeff_real = productivity_term(a_i, psi2, gamma, prod_mode, w)
            if coeff_real <= PROD_FLOOR + 1e-14:
                diag.real_floor_hits += 1
            diag.real_total_sim += 1

            Rev  = coeff_real * (k ** alpha)
            FE   = psi2

            # omega update with log link (den floored at 2)
            ratio = max(Rev / Rev_M, RATIO_EPS)
            den_raw = phi + tau + chi * np.log(ratio)
            if den_raw < DEN_FLOOR - 1e-12:
                diag.den_floor_hits_sim += 1
            den = max(den_raw, DEN_FLOOR)
            omega_new = 1.0 / den

            # bound hits (relative to grid)
            diag.omega_updates_sim += 1
            if omega_new <= omega_grid[0] * (1 + 1e-12):
                diag.omega_at_min_sim += 1
            if omega_new >= omega_grid[-1] * (1 - 1e-12):
                diag.omega_at_max_sim += 1

            if not (np.isfinite(Rev) and np.isfinite(omega_new)):
                diag.nans_sim += 1

            # record row
            rows.append({
                "firm": i,
                "time": t,
                "Rev":  float(Rev),
                "FE":   float(FE),
                "K":    float(k),
                "Omega":float(omega[i]),
                "A":    float(a_i),
                "lnA":  float(np.log(max(a_i, 1e-300))),
            })

            # advance state
            omega[i] = omega_new

    # Print only if something was hit / useful
    diag.report(label="SIM")

    # Build tidy panel DataFrame
    df = pd.DataFrame(rows).sort_values(["firm","time"]).reset_index(drop=True)
    return df, diag

### Evaluation functions

In [6]:
# ==========================
# Moments & regressions (DataFrame)
# ==========================

def median_k_alpha(a_grid, omega_grid, k_grid, policies, mu_lnA, alpha):
    A_med = np.exp(mu_lnA)
    ia = int(np.argmin(np.abs(a_grid - A_med)))
    omega_med = float(np.median(omega_grid))
    iw = int(np.argmin(np.abs(omega_grid - omega_med)))
    k_M = k_grid[policies[ia, iw]]
    return k_M ** alpha

def winsorize_series(s, p=0.01):
    """
    Winsorize a pandas Series at both tails by p.
    Caps values below p-quantile and above (1-p)-quantile.
    """
    lo, hi = s.quantile([p, 1 - p])
    return s.clip(lower=lo, upper=hi)


def compute_key_moments_df(df, w, rev_floor=1e-12, fe_floor=1e-12, winsor_p=0.00):
    """
    Regression 1a (raw):      logRev_t ~ FE + lnA
    Regression 1b (raw):      logRev_t ~ log(FE) + lnA
    Regression 2a (raw):      FE       ~ logRev_{t-1}
    Regression 2b (raw):      log(FE)  ~ logRev_{t-1}

    Regression 1a (winsor):   logRev_t ~ FE_w + lnA
    Regression 1b (winsor):   logRev_t ~ log(FE_w) + lnA
    Regression 2a (winsor):   FE_w     ~ logRev_{t-1}
    Regression 2b (winsor):   log(FE_w)~ logRev_{t-1}

    Returns a dict m containing:
      - main keys (without suffix):     winsorized regression moments
      - *_raw keys:                     raw regression moments
      - fitted models for both:         _res* and _res*_raw
    """

    df = df.copy()

    # --- Construct logRev and lag ---
    df["logRev"] = np.log(np.clip(df["Rev"].values, rev_floor, None))
    df["logRev_lag"] = df.groupby("firm")["logRev"].shift(1)

    # --- FE: raw and winsorized ---
    df["FE_raw"] = df["FE"]
    df["FE_w"]   = winsorize_series(df["FE"], p=winsor_p)

    # logFE for both versions
    df["logFE_raw"] = np.log(np.clip(df["FE_raw"].values, fe_floor, None))
    df["logFE_w"]   = np.log(np.clip(df["FE_w"].values,   fe_floor, None))

    # --- w * k_M^alpha / Rev_M moment (unchanged) ---
    kM_alpha = median_k_alpha(a_grid, omega_grid, k_grid, policies, MU_LNA, ALPHA)
    moment_w_ratio = (w * kM_alpha) / REV_M
    print(f"[Moment] w*kM^alpha/Rev_M = {moment_w_ratio:.6g} (target 0.0016)")

    # ============= RAW REGRESSIONS =============

    # 1a_raw: logRev ~ FE_raw + lnA
    df1_raw = df.dropna(subset=["logRev", "FE_raw", "lnA"])
    X1_raw  = sm.add_constant(df1_raw[["FE_raw", "lnA"]].values)
    y1_raw  = df1_raw["logRev"].values
    res1_raw = sm.OLS(y1_raw, X1_raw, missing='drop').fit()

    # 1b_raw: logRev ~ logFE_raw + lnA
    df1b_raw = df.dropna(subset=["logRev", "logFE_raw", "lnA"])
    X1b_raw  = sm.add_constant(df1b_raw[["logFE_raw", "lnA"]].values)
    y1b_raw  = df1b_raw["logRev"].values
    res1_log_raw = sm.OLS(y1b_raw, X1b_raw, missing='drop').fit()

    # 2a_raw: FE_raw ~ logRev_lag
    df2_raw = df.dropna(subset=["FE_raw", "logRev_lag"])
    X2_raw  = sm.add_constant(df2_raw[["logRev_lag"]].values)
    y2_raw  = df2_raw["FE_raw"].values
    res2_raw = sm.OLS(y2_raw, X2_raw, missing='drop').fit()

    # 2b_raw: logFE_raw ~ logRev_lag
    df2b_raw = df.dropna(subset=["logFE_raw", "logRev_lag"])
    X2b_raw  = sm.add_constant(df2b_raw[["logRev_lag"]].values)
    y2b_raw  = df2b_raw["logFE_raw"].values
    res2_log_raw = sm.OLS(y2b_raw, X2b_raw, missing='drop').fit()

    # ============= WINSORIZED REGRESSIONS =============

    # 1a_w: logRev ~ FE_w + lnA
    df1_w = df.dropna(subset=["logRev", "FE_w", "lnA"])
    X1_w  = sm.add_constant(df1_w[["FE_w", "lnA"]].values)
    y1_w  = df1_w["logRev"].values
    res1_w = sm.OLS(y1_w, X1_w, missing='drop').fit()

    # 1b_w: logRev ~ logFE_w + lnA
    df1b_w = df.dropna(subset=["logRev", "logFE_w", "lnA"])
    X1b_w  = sm.add_constant(df1b_w[["logFE_w", "lnA"]].values)
    y1b_w  = df1b_w["logRev"].values
    res1_log_w = sm.OLS(y1b_w, X1b_w, missing='drop').fit()

    # 2a_w: FE_w ~ logRev_lag
    df2_w = df.dropna(subset=["FE_w", "logRev_lag"])
    X2_w  = sm.add_constant(df2_w[["logRev_lag"]].values)
    y2_w  = df2_w["FE_w"].values
    res2_w = sm.OLS(y2_w, X2_w, missing='drop').fit()

    # 2b_w: logFE_w ~ logRev_lag
    df2b_w = df.dropna(subset=["logFE_w", "logRev_lag"])
    X2b_w  = sm.add_constant(df2b_w[["logRev_lag"]].values)
    y2b_w  = df2b_w["logFE_w"].values
    res2_log_w = sm.OLS(y2b_w, X2b_w, missing='drop').fit()

    # ============= MOMENTS =============

    m = dict()

    # Keep FE distribution moments on RAW FE (closer to the model)
    m["logRev_mean"] = float(df["logRev"].mean())
    m["logRev_sd"]   = float(df["logRev"].std())
    m["FE_median"]   = float(df["FE_raw"].median())
    m["FE_mean"]     = float(df["FE_raw"].mean())
    m["FE_sd"]       = float(df["FE_raw"].std())

    # ----- RAW regression moments (store with *_raw suffix) -----
    m["beta_FE_in_logRev_raw"]        = float(res1_raw.params[1])
    m["beta_lnA_in_logRev_raw"]       = float(res1_raw.params[2])
    m["R2_reg1_raw"]                  = float(res1_raw.rsquared)
    m["n_reg1_raw"]                   = int(res1_raw.nobs)

    m["beta_logFE_in_logRev_raw"]     = float(res1_log_raw.params[1])
    m["beta_lnA_in_logRev_log_raw"]   = float(res1_log_raw.params[2])
    m["R2_reg1_log_raw"]              = float(res1_log_raw.rsquared)
    m["n_reg1_log_raw"]               = int(res1_log_raw.nobs)

    m["beta_logRevLag_in_FE_raw"]     = float(res2_raw.params[1])
    m["R2_reg2_raw"]                  = float(res2_raw.rsquared)
    m["n_reg2_raw"]                   = int(res2_raw.nobs)

    m["beta_logRevLag_in_logFE_raw"]  = float(res2_log_raw.params[1])
    m["R2_reg2_log_raw"]              = float(res2_log_raw.rsquared)
    m["n_reg2_log_raw"]               = int(res2_log_raw.nobs)

    # ----- WINSORIZED regression moments (MAIN keys, used by loss, etc.) -----
    m["beta_FE_in_logRev"]            = float(res1_w.params[1])
    m["beta_lnA_in_logRev"]           = float(res1_w.params[2])
    m["R2_reg1"]                      = float(res1_w.rsquared)
    m["n_reg1"]                       = int(res1_w.nobs)

    m["beta_logFE_in_logRev"]         = float(res1_log_w.params[1])
    m["beta_lnA_in_logRev_log"]       = float(res1_log_w.params[2])
    m["R2_reg1_log"]                  = float(res1_log_w.rsquared)
    m["n_reg1_log"]                   = int(res1_log_w.nobs)

    m["beta_logRevLag_in_FE"]         = float(res2_w.params[1])
    m["R2_reg2"]                      = float(res2_w.rsquared)
    m["n_reg2"]                       = int(res2_w.nobs)

    m["beta_logRevLag_in_logFE"]      = float(res2_log_w.params[1])
    m["R2_reg2_log"]                  = float(res2_log_w.rsquared)
    m["n_reg2_log"]                   = int(res2_log_w.nobs)

    # store fitted models
    # main (winsorized) versions keep the old names
    m["_res1"]      = res1_w
    m["_res1_log"]  = res1_log_w
    m["_res2"]      = res2_w
    m["_res2_log"]  = res2_log_w

    # raw versions with *_raw suffix
    m["_res1_raw"]      = res1_raw
    m["_res1_log_raw"]  = res1_log_raw
    m["_res2_raw"]      = res2_raw
    m["_res2_log_raw"]  = res2_log_raw

    m["w_ratio_median"] = moment_w_ratio

    return m


def print_key_moments(m):
    print("=== Key moments ===")
    print(f"{'logRev_mean:':>26}  {m['logRev_mean']:.4f}")
    print(f"{'logRev_sd:':>26}  {m['logRev_sd']:.4f}")
    print(f"{'FE_median:':>26}  {m['FE_median']:.4f}")
    print(f"{'FE_mean:':>26}  {m['FE_mean']:.4f}")
    print(f"{'FE_sd:':>26}  {m['FE_sd']:.4f}")
    print(f"{'beta_FE_in_logRev:':>26}  {m['beta_FE_in_logRev']:.4f}")
    print(f"{'beta_lnA_in_logRev:':>26}  {m['beta_lnA_in_logRev']:.4f}")
    print(f"{'R2_reg1:':>26}  {m['R2_reg1']:.4f}")
    print(f"{'beta_logFE_in_logRev:':>26}  {m['beta_logFE_in_logRev']:.4f}")
    print(f"{'R2_reg1_log:':>26}  {m['R2_reg1_log']:.4f}")
    print(f"{'n_reg1_log:':>26}  {m['n_reg1_log']}")
    print(f"{'beta_logRevLag_in_FE:':>26}  {m['beta_logRevLag_in_FE']:.4f}")
    print(f"{'R2_reg2:':>26}  {m['R2_reg2']:.4f}")
    print(f"{'n_reg2:':>26}  {m['n_reg2']}")
    # NEW: log(FE) ~ logRev_lag
    print(f"{'beta_logRevLag_in_logFE:':>26}  {m['beta_logRevLag_in_logFE']:.4f}")
    print(f"{'R2_reg2_log:':>26}  {m['R2_reg2_log']:.4f}")
    print(f"{'n_reg2_log:':>26}  {m['n_reg2_log']}")
    print()


def print_reg_summaries(m):
    res1     = m["_res1"]
    res1_log = m["_res1_log"]
    res2     = m["_res2"]
    res2_log = m["_res2_log"]

    # Label regressors
    res1.model.data.xnames     = ["const", "FE", "lnA"]
    res1_log.model.data.xnames = ["const", "logFE", "lnA"]
    res2.model.data.xnames     = ["const", "logRev_lag"]
    res2_log.model.data.xnames = ["const", "logRev_lag"]

    print("=== Regression 1a: logRev ~ FE + lnA ===")
    print(res1.summary().tables[1].as_text()); print()

    print("=== Regression 1b: logRev ~ log(FE) + lnA ===")
    print(res1_log.summary().tables[1].as_text()); print()

    print("=== Regression 2a: FE ~ logRev_lag ===")
    print(res2.summary().tables[1].as_text()); print()

    print("=== Regression 2b: log(FE) ~ logRev_lag ===")
    print(res2_log.summary().tables[1].as_text()); print()


def evaluate_targets_and_loss(m, targets):
    """
    targets = {
      'logRev_mean': 6.63,
      'logRev_sd': 1.72,
      'beta_FE_in_logRev': -0.87,
      'beta_logRevLag_in_FE': -0.0055
    }
    """
    rows = [
        ("Mean log revenue",          "logRev_mean"),
        ("SD log revenue",            "logRev_sd"),
        ("β_logFE in logRev (reg1)",     "beta_logFE_in_logRev"),
        ("β_logRevLag in log FE (reg2)",  "beta_logRevLag_in_logFE"),
        ("w * k_M^alpha / Rev_M",     "w_ratio_median"),  # <-- NEW ROW
    ]
    print("=== Targets vs Model (and loss terms) ===")
    print(f"{'Moment':35} {'Target':>12} {'Model':>12} {'Model/Target':>14} {'|ratio-1|':>14}")
    loss_terms = []
    for label, key in rows:
        tgt = float(targets[key])
        mdl = float(m[key])
        ratio = mdl / tgt
        term = abs(ratio - 1.0)
        loss_terms.append(term)
        print(f"{label:35} {tgt:12.6f} {mdl:12.6f} {ratio:14.6f} {term:14.6f}")
    total_loss = float(np.sum(loss_terms))
    print(f"{'Total loss':35} {'':12} {'':12} {'':14} {total_loss:14.6f}\n")
    return total_loss

## Execution

### Main

In [7]:
# =================================
# Global constants / numeric floors
# =================================
BETA       = 1.0 / 1.04  # discount factor (r ~ 0.04)
PROD_FLOOR = 0.001         # min productivity term (flow & realized)
DEN_FLOOR  = 1.6         # min denominator in omega mapping
RATIO_EPS  = 1e-12       # floor inside log(Rev/Rev_M)
VFI_TOL    = 1e-6        # convergence tolerance for VFI
VFI_MAXIT  = 500         # max VFI iterations


if __name__ == "__main__":
    # --- Parameters you can tweak ---
    MU_LNA  = 3.77
    SD_LNA  = 1.05
    ALPHA   = 1/3
    r       = 0.04
    PHI     = 142.85
    TAU     = 293.77
    REV_M   = 658
    w = 0.056
    #PROD_MODE = "additive"   # or  "multiplicative"
    PROD_MODE = "multiplicative"
    if PROD_MODE == "additive":
        GAMMA   = 1.19 # additive
        CHI     = 117.0
    else:
        GAMMA   = 1.47    # multiplicative
        CHI     = 126


    # --- Build grids ---
    a_grid, omega_grid, k_grid = make_grids(
        MU_LNA, SD_LNA,
        N_a=51, N_w=201, N_k=101,
        alpha=ALPHA, r=r,
        phi=PHI, tau=TAU, chi=CHI, Rev_M=REV_M,
        gamma=GAMMA, prod_mode=PROD_MODE, w = w,
        k_min=1, logRev_cap=9.00, a_ref_q=0.8,
        omega_min = 1e-7, omega_max = 0.5
    )

    # --- Solve policies ---
    values, policies, diag_vfi = solve_policy(
        a_grid, omega_grid, k_grid,
        alpha=ALPHA, r=r, phi=PHI, tau=TAU, chi=CHI, Rev_M=REV_M,
        gamma=GAMMA, prod_mode=PROD_MODE, w = w
    )

    # --- Simulate panel (now returns a DataFrame) ---
    df_panel, diag_sim = simulate_panel(
        N_firms=1000, T=100,  # bump T for cleaner lag regression
        a_grid=a_grid, omega_grid=omega_grid, k_grid=k_grid, policies=policies,
        mu_lnA=MU_LNA, sd_lnA=SD_LNA, alpha=ALPHA,
        phi=PHI, tau=TAU, chi=CHI, Rev_M=REV_M,
        gamma=GAMMA, prod_mode=PROD_MODE, w = w,
        seed=2025
    )

    # --- Moments, regressions, loss ---
    moments = compute_key_moments_df(df_panel, w)
    print_key_moments(moments)

    print("β_logFE in logRev (RAW):       ", moments["beta_logFE_in_logRev_raw"])
    print("β_logFE in logRev (WINSORIZED):", moments["beta_logFE_in_logRev"])

    print("β_logRevLag in logFE (raw vs win):",
          moments["beta_logRevLag_in_logFE_raw"],
          moments["beta_logRevLag_in_logFE"])

    print_reg_summaries(moments)

    targets = {
        "logRev_mean": 6.63,
        "logRev_sd": 1.72,
        "beta_logFE_in_logRev": -0.0163,
        "beta_logRevLag_in_logFE": -0.41,
        "w_ratio_median": 0.0016,  # NEW
    }
    _ = evaluate_targets_and_loss(moments, targets)

k_max = 491789.31892578094
[Grids] A_ref(q=0.80)=103 | k_max=4.92e+05
[Grids] Rev_ref=408 | omega_min=1.000e-07
[Grids] worst Rev=32.9 | omega_max=5.000e-01
[VFI] omega-denominator floored (VFI): 3.71% (18814794/507362592)
[VFI] policy grid hits: k_min=0.00%, k_max=0.00% (of 5023392 states)
[SIM] realized productivity floored: 0.06% (64/100000)
[SIM] omega bound hits (SIM): min=0.00%, max=0.45% (of 100000 updates)
[Moment] w*kM^alpha/Rev_M = 0.00158901 (target 0.0016)
=== Key moments ===
              logRev_mean:  6.6708
                logRev_sd:  1.5945
                FE_median:  0.0011
                  FE_mean:  0.0060
                    FE_sd:  0.0679
        beta_FE_in_logRev:  -3.0916
       beta_lnA_in_logRev:  1.4686
                  R2_reg1:  0.9907
     beta_logFE_in_logRev:  -0.0122
              R2_reg1_log:  0.9740
               n_reg1_log:  100000
     beta_logRevLag_in_FE:  -0.0080
                  R2_reg2:  0.0349
                   n_reg2:  99000
  beta_logRevLa

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
import pickle
import os

# Define the path to save the file
save_path = os.path.join(project_dir, 'Output/model_SMM5.pkl')

# Create a dictionary of the variables you want to save.
# I've included the main grids, policies, values, parameters, and results.
saved_variables = {
    'a_grid': a_grid,
    'omega_grid': omega_grid,
    'k_grid': k_grid,
    'policies': policies,
    'values': values,
    'df_panel': df_panel,
    'moments': moments,
    'targets': targets,
    'MU_LNA': MU_LNA,
    'SD_LNA': SD_LNA,
    'ALPHA': ALPHA,
    'r': r,
    'PHI': PHI,
    'TAU': TAU,
    'CHI': CHI,
    'REV_M': REV_M,
    'w': w,
    'GAMMA': GAMMA,
    'PROD_MODE': PROD_MODE,
    'BETA': BETA,
    'PROD_FLOOR': PROD_FLOOR,
    'DEN_FLOOR': DEN_FLOOR,
    'RATIO_EPS': RATIO_EPS,
    'VFI_TOL': VFI_TOL,
    'VFI_MAXIT': VFI_MAXIT
}

# Save the dictionary to a pickle file
with open(save_path, 'wb') as f:
    pickle.dump(saved_variables, f)

print(f"All selected variables saved to: {save_path}")

All selected variables saved to: /content/drive/MyDrive/Data_GDP_OV/Output/model_SMM5.pkl
